## Current Plan

#### Radar Chart - %win by being dominant in a number of different stats

#### Interactive chart - scatter perhaps. Points scored by multiple drop down lists. Size of point indicative of tries scored, colour perhaps by country? Perhaps split into years? Or just this year?


###### Animated graph - (splitviolin?), x axis is win/lose; y is choice of different stats - do they correlate to winning? Show correlation coeff somehow. Color coeff somehow?

###### Animated graph (possibly violin?). Animation over years. x = win_or_lose, y: can choose between kicks, kick_abs and kick_ratio
###### OR animated graph. x= kick_margin, y = score, color = tries, animation_frame = years

In [117]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import statsmodels.api as sm
from jupyter_dash import JupyterDash
from dash import Dash, html, dcc, Input, Output
import dash_bootstrap_components as dbc

In [118]:
df = pd.read_csv('sixnationsstats_V3.csv')
df['score_margin'] = df['score'] - df['opp_score']
df['abs_score_margin'] = df['score_margin'].abs()
df['kick_margin'] = df['kicks'] - df['opp_kicks']
df['cards'] = df['yellow_cards'] + df['red_cards']
df['set_piece_success%'] = (df['scrums_won%'] + df['lineouts_won%']) / 2
df['opp_set_piece_success%'] = (df['opp_scrums_won%'] + df['opp_lineouts_won%']) / 2
df['total_kicks'] = df['kicks'] + df['opp_kicks']
df['total_tries'] = df['tries'] + df['opp_tries']

#dropna tries
df = df.dropna(subset=['tries'])
df = df.dropna(subset=['score'])

#list(df.columns.values)
df.head()

,year,location,team,score,penalties,tries,conversions,dropgoals,territory%,possession%,...,opp_kicks,opp_kick_ratio,score_margin,abs_score_margin,kick_margin,cards,set_piece_success%,opp_set_piece_success%,total_kicks,total_tries
0,2015,home,Wales,16.0,2.0,1.0,1.0,1.0,41.0,51.0,...,32.0,27.027,-5.0,5.0,0.0,1.0,73.0,91.0,64.0,3.0
1,2015,away,England,21.0,3.0,2.0,1.0,0.0,59.0,49.0,...,32.0,26.316,5.0,5.0,0.0,0.0,91.0,73.0,64.0,3.0
2,2015,home,Italy,3.0,1.0,0.0,0.0,0.0,48.0,37.0,...,28.0,14.493,-23.0,23.0,-13.0,1.0,75.0,93.0,43.0,2.0
3,2015,away,Ireland,26.0,4.0,2.0,2.0,0.0,51.0,63.0,...,15.0,10.417,23.0,23.0,13.0,0.0,93.0,75.0,43.0,2.0
4,2015,home,France,15.0,5.0,0.0,0.0,0.0,53.0,54.0,...,25.0,19.231,7.0,7.0,4.0,0.0,95.0,82.5,54.0,1.0


In [119]:
key_stats = ['linebreaks', 'carries', 'to_won', 'tackles_made', 'tackle_success_rate%',
                   'kicks', 'kick_ratio', 'set_piece_success%', 'penalties_won', 'kick_ratio'] 

In [120]:
for stat in key_stats:
    new_col_name = stat + '_dom'
    opp_stat = 'opp_' + stat
    df[new_col_name] =  df[stat] > df[opp_stat]

df.head()

#df.assign(linebreaks_dom = lambda x: df['linebreaks'] > df['opp_linebreaks'])

,year,location,team,score,penalties,tries,conversions,dropgoals,territory%,possession%,...,total_tries,linebreaks_dom,carries_dom,to_won_dom,tackles_made_dom,tackle_success_rate%_dom,kicks_dom,kick_ratio_dom,set_piece_success%_dom,penalties_won_dom
0,2015,home,Wales,16.0,2.0,1.0,1.0,1.0,41.0,51.0,...,3.0,False,False,True,False,False,False,False,False,False
1,2015,away,England,21.0,3.0,2.0,1.0,0.0,59.0,49.0,...,3.0,True,True,False,True,True,False,True,True,False
2,2015,home,Italy,3.0,1.0,0.0,0.0,0.0,48.0,37.0,...,2.0,False,False,False,True,True,False,False,False,False
3,2015,away,Ireland,26.0,4.0,2.0,2.0,0.0,51.0,63.0,...,2.0,True,True,True,False,False,True,True,True,True
4,2015,home,France,15.0,5.0,0.0,0.0,0.0,53.0,54.0,...,1.0,False,True,False,False,True,True,False,True,True


In [121]:
def win_lose(margin):
    if margin > 0:
        return 'won'
    elif margin < 0:
        return 'lost'
    else:
        return None

def coded_win_lose(win_lose):
    if win_lose == 'won':
        return 1
    elif win_lose == 'lost':
        return 0
    
df['win_or_lose'] = df['score_margin'].map(win_lose)
df['coded_win_or_lose'] = df['win_or_lose'].map(coded_win_lose)

In [122]:
kick_ratio = df[df['kick_ratio'] < 50] # removing kick ratio outliers
kick_ratio = df[(df['kick_ratio'] < 50) & (df['win_or_lose'] == 'won')]
len(kick_ratio)

90

In [123]:
fig = px.scatter(df, y='tries', x='kicks', trendline='ols', trendline_scope='overall', color='team')
fig.show()

In [124]:
#Correlations
key_stats = ['linebreaks', 'to_won', 'to_lost', 'penalties_won', 'tackles_made', 'tackle_success_rate%',
                   'kicks', 'kick_ratio', 'kick_margin', 'set_piece_success%']

correlations = {}

for stat in key_stats:
    correlations[stat] = df['coded_win_or_lose'].corr(df[stat])

correlations

{'linebreaks': 0.30059347239014406,
 'to_won': 0.2548587936571141,
 'to_lost': -0.2840185956157666,
 'penalties_won': 0.10462708275359615,
 'tackles_made': 0.043543039694966,
 'tackle_success_rate%': 0.011882365455499392,
 'kicks': 0.32822920562257685,
 'kick_ratio': 0.2884741357136207,
 'kick_margin': 0.6276717671550276,
 'set_piece_success%': 0.07612761438376947}

In [125]:
fig = go.Figure()

fig.add_trace(
    go.Violin(x=df['win_or_lose'][ df['win_or_lose'] == 'lost'],
             y = df['kick_ratio'][ df['win_or_lose'] == 'lost'],
             name='Lost',
             line_color='darkred',
             fillcolor='red',
             opacity=0.6,
             side='negative',
             scalegroup='No',
             box_visible=True))

fig.add_trace(
    go.Violin(x=df['win_or_lose'][ df['win_or_lose'] == 'won'],
             y = df['kick_ratio'][ df['win_or_lose'] == 'won'],
             name='Won',
             line_color='darkgreen',
             fillcolor='green',
             opacity=0.6,
             side='positive',
             scalegroup='Yes',            
             box_visible=True))


fig.update_layout(violingap=0, violingroupgap=0, violinmode='overlay')
#### TODO WORK OUT HOW TO OVERLAY!
fig.show()


In [126]:
kick_ratio = df[df['kick_ratio'] < 50] # removing kick ratio outliers
kick_ratio = df[(df['kick_ratio'] < 50) & (df['win_or_lose'] == 'won')]

fig = go.Figure()

fig.add_trace(
    go.Violin(y = df['kick_margin'][(df['kick_margin'] > -25) & (df['win_or_lose'] == 'lost')],
             name='Lost',
             line_color='darkred',
             fillcolor='red',
             opacity=0.6,
             side='negative',
             scalegroup='No',
             box_visible=True,
             x0 = 1))

fig.add_trace(
    go.Violin(y = df['kick_margin'][(df['kick_margin'] < 25) & (df['win_or_lose'] == 'won')],
             name='Won',
             line_color='darkgreen',
             fillcolor='green',
             opacity=0.6,
             side='positive',
             scalegroup='Yes',            
             box_visible=True,
             x0=1))


fig.update_layout(violingap=0, violinmode='overlay')
#### TODO WORK OUT HOW TO OVERLAY! x0 feels a bodge
fig.show()

##proof of concept!

In [127]:
fig = px.violin(df, y='kick_margin', x='win_or_lose', color='win_or_lose',  box=True, violinmode='overlay')

print(fig)
fig.show()

Figure({
    'data': [{'alignmentgroup': 'True',
              'box': {'visible': True},
              'hovertemplate': 'win_or_lose=%{x}<br>kick_margin=%{y}<extra></extra>',
              'legendgroup': 'lost',
              'marker': {'color': '#636efa'},
              'name': 'lost',
              'offsetgroup': 'lost',
              'orientation': 'v',
              'scalegroup': 'True',
              'showlegend': True,
              'type': 'violin',
              'x': array(['lost', 'lost', 'lost', 'lost', 'lost', 'lost', 'lost', 'lost', 'lost',
                          'lost', 'lost', 'lost', 'lost', 'lost', 'lost', 'lost', 'lost', 'lost',
                          'lost', 'lost', 'lost', 'lost', 'lost', 'lost', 'lost', 'lost', 'lost',
                          'lost', 'lost', 'lost', 'lost', 'lost', 'lost', 'lost', 'lost', 'lost',
                          'lost', 'lost', 'lost', 'lost', 'lost', 'lost', 'lost', 'lost', 'lost',
                          'lost', 'lost', 'lost',

In [128]:
fig = px.scatter(df, x='year', y='kick_ratio', color='win_or_lose', size='abs_score_margin', trendline="ols")
fig.show()

In [129]:
fig = px.violin(df, y='kick_margin', x='win_or_lose', color='win_or_lose',  box=True)
fig.show()

In [130]:
kick_ratio = df[df['kick_ratio'] < 50] # removing kick ratio outliers


In [131]:
fig = px.violin(df, y='kick_ratio', x='win_or_lose', color='win_or_lose', box=True, animation_frame='year')
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1000
fig.show()

In [132]:
df['tries'].isnull().sum()

0

In [133]:
fig = px.scatter(df, y='score_margin', x='kick_margin', size='tries', color = 'team',
                 animation_frame='year', trendline='ols', trendline_scope = 'overall')
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1000
fig.show()

In [134]:
fig = px.scatter(df, x='kicks', y='score', color='win_or_lose', size='abs_score_margin', animation_frame='year')
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1000
fig.show()

In [135]:
tackle_success_noout = df[df['tackle_success_rate%'] > 50] # removing kick ratio outliers

fig = px.scatter(tackle_success_noout, x='score', y='tackle_success_rate%', color='win_or_lose', size='abs_score_margin', trendline="ols")
fig.show()

In [136]:
fig = px.violin(tackle_success_noout, x='win_or_lose', y='tackle_success_rate%', box=True)
fig.show()

In [137]:
# 'metres_after_contact', 'linebreaks', 'to_won', 'to_lost', 'penalties_conceded', 'location', 'tackle_success_rate%', 'kicks', 'kick_ratio',

fig = px.violin(df, x='win_or_lose', y='penalties_won', color='win_or_lose')
fig.show()

In [138]:
cards = df[df['cards'] > 0]

fig = px.scatter(cards, x='score', y='score_margin', color='win_or_lose')
#fig.show()

In [139]:
fig = px.violin(df, x='win_or_lose', y='kick_margin', animation_frame='year', color = 'win_or_lose', box=True)
fig.show()

In [140]:
fig = px.bar(df, x='year', y='tries', color='team')
fig.show()

In [141]:
dimensions = ['linebreaks', 'tackle_success_rate%', 'to_won', 'to_lost']

dimensions2 = ['penalties_won', 'to_won', 'to_lost', 'location']#, 'kicks', 'kick_ratio']
fig = px.scatter_matrix(tackle_success_noout, dimensions=dimensions2, color='win_or_lose')
fig.show()

In [142]:
fig = px.scatter(df, x='linebreaks', y='score', size='tries', trendline='ols')
fig.show()

In [143]:
fig = px.scatter(df, x='to_won', y='score', color='tries', trendline='ols')
fig.show()

In [144]:
fig = px.scatter(df, x='set_piece_success%', y='score', color='win_or_lose', trendline='ols')
fig.show()

In [155]:
stats_to_compare =  ['linebreaks_dom', 'kick_ratio_dom', 'to_won_dom', 'tackles_made_dom',
                     'tackle_success_rate%_dom', 'kicks_dom', 'set_piece_success%_dom', 'penalties_won_dom']

win_percentage = {}

for stat in stats_to_compare:
    df_of_stat = df[stat]
    win_percentage[stat] = round(len(df[(df[stat] == True) & (df['win_or_lose'] == 'won')]) / df_of_stat.sum() * 100, 1)

win_percentage_dict = {
    'theta' : list(win_percentage.keys()),
    'r' : list(win_percentage.values())
    }

win_percentage_dict    

{'theta': ['linebreaks_dom',
  'kick_ratio_dom',
  'to_won_dom',
  'tackles_made_dom',
  'tackle_success_rate%_dom',
  'kicks_dom',
  'set_piece_success%_dom',
  'penalties_won_dom'],
 'r': [69.0, 70.5, 66.7, 51.6, 53.7, 82.4, 64.4, 55.1]}

In [156]:
stat_list_for_x = {'linebreaks':['Linebreaks','the Number of Linebreaks'],
                   'kick_ratio':['Kick to Pass Ratio','the Kick to Pass Ratio'],
                   'kicks':['Kicks','the Number of Kicks'],
                   'tackles_made':['Tackles','the Number of Tackles'],                        
                   'penalties_won':['Penalties Won','the Number of Penalties Won'],
                   'set_piece_success%':['Own Set Piece Retention (%)','Retaining Posession on Own Set Piece'],
                   'to_won': ['Turnovers Won','the Number of Turnovers Won'],
                  }

win_percentage = {}

for stat, title in stat_list_for_x.items():
    dom_stat = stat + '_dom'
    df_of_stat = df[dom_stat]
    win_percentage[title[0]] = round(len(df[(df[dom_stat] == True) & (df['win_or_lose'] == 'won')]) / df_of_stat.sum() * 100, 1)

win_percentage_dict = {
    'theta' : list(win_percentage.keys()),
    'r' : list(win_percentage.values())
    }

r = win_percentage.values()
theta = win_percentage.keys()

In [157]:
fig = px.bar_polar(pd.DataFrame(win_percentage_dict), r='r', theta='theta', range_r=[0, 100], template='plotly_dark', 
                   color='r', color_discrete_sequence = 'YlGn')
fig.show()

#### The below is a starter. Need to refine background colours etc. Pretty it up. Decide order to show in. Sort out names. Change hover font colour and size to be consistent (and work with background)

In [158]:
def create_polar_bar():
    color_no = [int(x*10) for x in r]

    fig = go.Figure(data=go.Barpolar(
        r = win_percentage_dict.get('r'),
        theta = win_percentage_dict.get('theta'),
        marker= dict(cmax=850,
                     cmin=350,
                     color=color_no,
                     colorscale='Greens',
                     line_width=0),
        opacity=1,
        hovertemplate= ['%{theta}: <b>%{r}</b>%<extra></extra>' for item in theta],
        hoverlabel=dict(bgcolor='white',
                        font=dict(color="#0c0c28")),
    ))

    fig.update_layout(
        title = dict(text='Win percentage when a team dominates in a given statistic'),
        title_x = 0.5,
        polar = dict(radialaxis = dict(range=[0, 100]),
                     bgcolor= "#11123b"),
        font=dict(color="#FFFFFF"),
        paper_bgcolor= "#11123b")

    return fig

polar_figure = create_polar_bar()

polar_figure.show()

##### Interactive chart - scatter perhaps. Points scored by multiple drop down lists. Size of point indicative of tries scored, colour perhaps by country?

In [159]:
color_scheme = {
    'England' : '#ffffff',
    'France' : '#0000c0',
    'Ireland' : '#00845c',
    'Italy' : '#0076ff',
    'Scotland' : '#002a66',
    'Wales' : '#e32a2d',
}
# Wales #ff0000
# Scotland '#000080
#Italy #0052b1

Proof of concept. Do the below by with x changing to kicks, kick_ratio, linebreaks, tackles_made. 

In [160]:
fig = px.scatter(df, y='score', x='kicks', size='tries', color = 'team', trendline='ols', trendline_scope = 'overall')
#fig.show()


In [161]:
marker_sizes = {0: 8,
                1: 9.5,
                2: 11,
                3: 12.5,
                4: 14,
                5: 15.5,
                6: 17,
                7: 18.5,
                8: 20,
                9: 21.5}

In [165]:
def create_graph(stat, title):

    fig = go.Figure()
   
    for team, color in color_scheme.items():
        df_team = df[df['team'] == team]
        sizes = df_team['tries'].map(marker_sizes).fillna(10)
        
        
        trace = go.Scatter(
                x= df_team[stat],
                y= df_team['score_margin'],
                customdata= np.stack((df_team['team'], df_team['opp_team'], df_team['year'], df_team['tries']), axis=1),
                name = team,
                mode='markers',
                marker=dict(size= sizes, 
                            color= color,
                            opacity= 0.9,
                            ),  # title for the colorbar
                textfont=dict(color='#FFFFFF'),
                hoverlabel=dict(bgcolor='white',
                                font=dict(color="#0c0c28")),
                hovertemplate='<b>%{customdata[0]}</b> against %{customdata[1]}, %{customdata[2]}<br>'\
                                '<b>%{customdata[3]}</b> tries<extra></extra>',
        )
        
        fig.add_trace(trace)
    
    #Add a trendline    
    x = sm.add_constant(df[stat])
    y = df['score_margin'].values.reshape(-1, 1)
    model = sm.OLS(y, x).fit()
    trendline_R2 = model.rsquared
    m = model.params[1]
    b = model.params[0]

    trendline_x = np.array([df[stat].min(), df[stat].max()])
    trendline_y = m * trendline_x + b
    
    fig.add_trace(go.Scatter(x=trendline_x, y=trendline_y, mode='lines', 
                             line=dict(color='#d97f43', width=2),
                             name= "Overall Trend",
                             hoverinfo='skip'
                             ))
    
    fig.update_layout(title=f'Impact of {title[1]} has on the Score',
                      title_x = 0.5,
                      font=dict(color="#FFFFFF"),
                      paper_bgcolor="#11123b",
                      plot_bgcolor= "#0c0c28",
                     )
    
    x_axis_min = 0
    if df[stat].min() == 0:
        x_axis_min = -1.5
    elif df[stat].min() > 15:
        x_axis_min = df[stat].min()-5
        
    x_axis_max = df[stat].max()*1.05
        
    fig.update_xaxes(title_text=title[0].title(),
                     range=[x_axis_min, x_axis_max]
                    )
    fig.update_yaxes(title_text='Score Differential',
                     range=[-60,60])
    
    

    return fig

fig = create_graph('kicks', 'ciciau')
fig.show()

In [163]:
for stat, title in stat_list_for_x.items():
    fig = create_graph(stat, title)
    fig.show()

# Creating the Dash App

In [164]:
app = JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP])

colors = {
    'background': '#11123b',
    'text': '#FFFFFF',
    'width': '99vw'
}

header = 'Do teams kick too much? Is kicking making rugby boring?'
pre_text = 'There\'s a lot of disscussion about kicking in the modern game with some pundits arguing that'\
            ' kicking is killing the game. Have a look through the stats yourself to see if they\'re right.'
first_text = 'Note: A team is dominant in a stat when they simply outnumber or outscore their opposition in that stat. i.e. they' \
             ' made more tackles or kicked more times.'
second_text = '"Okay, teams that kick more win more. But I don\'t want to watch a boring match. I want excitement'\
            ' - lots of tries and high scores!"'
third_text = 'Have a look for yourself - which of these stats tend towards a higher score with more tries?'
fourth_text = 'Note: The bigger the circle, the more tries that the team has scored. You can hover over to see which match'\
            ' it relates to.'



app.layout = dbc.Container(
    style={'backgroundColor': colors['background'],
           'color': colors['text'],           
          },
    children=[
        html.Div(className= 'm-2',
                children=[
                html.H1([header], style={'text-align': 'center'}, className='m-4 p-1'),
                html.Div(children=[
                    html.H6(pre_text),
                    html.P(html.Small(html.I(first_text)))
                ]),

                dcc.Graph(id='polar',
                         figure=polar_figure),

                html.Div(children=[
                    html.P(html.I(html.B(second_text)), style={'text-align': 'center'}),
                    html.H6(third_text),
                    html.P(html.Small(html.I(fourth_text)))
                ]),


                dbc.Row(
                    [
                        dbc.Col([
                            dbc.Label('Select a Stat:', html_for='scatter_dropdown'),
                            dcc.Dropdown(id='scatter_dropdown',
                                
                                options=[{'label': title[0], 'value': stat} for stat, title in stat_list_for_x.items()],
                                value = 'kicks',
                                optionHeight= 45,
                                searchable = False,
                                placeholder = 'Please select a stat',
                                clearable = False,
                                style={'width': "95%", 'color':colors['background']})
                        ], width=2),

                        dbc.Col([
                            dcc.Graph(id='scatter'),
                        ], width=10),

                    ], align='center'
                ),

            ])
    ])

@app.callback(
    Output(component_id='scatter', component_property='figure'),
    [Input(component_id='scatter_dropdown', component_property='value')]
)
def create_graph(stat):
    graph_df = df
    
    
    title = stat_list_for_x.get(stat)


    fig = go.Figure()
   
    for team, color in color_scheme.items():
        df_team = graph_df[graph_df['team'] == team]
        sizes = df_team['tries'].map(marker_sizes).fillna(10)
        
        
        trace = go.Scatter(
                x= df_team[stat],
                y= df_team['score'],
                customdata= np.stack((df_team['team'], df_team['opp_team'], df_team['year'], df_team['tries']), axis=1),
                name = team,
                mode='markers',
                marker=dict(size= sizes, 
                            color= color,
                            opacity= 0.9,
                            ),  # title for the colorbar
                hoverlabel=dict(bgcolor='white',
                                font=dict(color="#0c0c28")),
                hovertemplate='<b>%{customdata[0]}</b> against %{customdata[1]}, %{customdata[2]}<br>'\
                                '<b>%{customdata[3]}</b> tries<extra></extra>',
        )
        
        fig.add_trace(trace)
    
    #Add a trendline    
    x = sm.add_constant(df[stat])
    y = df['score'].values.reshape(-1, 1)
    model = sm.OLS(y, x).fit()
    trendline_R2 = model.rsquared
    m = model.params[1]
    b = model.params[0]

    trendline_x = np.array([df[stat].min(), df[stat].max()])
    trendline_y = m * trendline_x + b
    
    fig.add_trace(go.Scatter(x=trendline_x, y=trendline_y, mode='lines', 
                             line=dict(color='#d97f43', width=2),
                             name= "Overall Trend",
                             hoverinfo='skip'
                             ))
    
    fig.update_layout(title=f'Impact of {title[1]} has on the Score', 
                      title_x= 0.5,
                      font=dict(color="#FFFFFF"),
                      paper_bgcolor="#11123b",
                      plot_bgcolor= "#0c0c28",
                     )
    
    x_axis_min = 0
    if df[stat].min() == 0:
        x_axis_min = -1.5
    elif df[stat].min() > 15:
        x_axis_min = df[stat].min()-5
        
    x_axis_max = df[stat].max()*1.05
        
    fig.update_xaxes(title_text=title[0].title(), 
                     range=[x_axis_min, x_axis_max]
                    )
    fig.update_yaxes(title_text='Score',
                     range=[-5,75]
                    )
    
    

    return (fig)

if __name__ == '__main__':
    app.run_server(debug=True)
    

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/
